<a href="https://colab.research.google.com/github/thxsxth/POMDP_RLSepsis/blob/master/Distributional_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd 'drive/My Drive/sepsis3-cohort'


/content/drive/.shortcut-targets-by-id/1Dn0AjHyN50Kv8tcF0f2bQsdPbRYa_49h/sepsis3-cohort


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cuda' if torch.cuda.is_available() else 'cpu'
print('Done')

Done


In [ ]:
df=pd.read_csv('RL_new.csv')
 df=df.iloc[:,1:]
print('Loaded')

Loaded


In [ ]:
df.head()

,DBP,HR,MBP,RR,SOFA,SpO2,SBP,TempC,cardio,cns,liver,renal,AG,BICARB,CREAT,CL,GLUC,HCT,HB,PLAT,K,Na,BUN,WBC,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,R,C,T,SV,age,M/F,weight,Vaso,Fluids,Rewards,Pat
0,46.0,118.0,62.0,26.0,0.0,94.0,109.0,36.388889,0.0,0.0,0.0,0.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.463221,1.030419,0.419510,65.685165,82.0,0.0,75.1,0.0,0.0,-0.250,262147
1,41.0,111.0,58.0,23.0,2.0,95.0,109.0,36.388889,1.0,0.0,0.0,1.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.461611,0.946364,0.446352,66.568794,82.0,0.0,75.1,0.0,0.0,-0.375,262147
2,44.0,108.0,62.0,19.0,5.0,98.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.509343,0.863491,0.458074,66.941925,82.0,0.0,75.1,0.0,0.0,-0.025,262147
3,41.0,96.0,60.0,20.0,5.0,96.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.554594,0.871751,0.518681,68.092636,82.0,0.0,75.1,0.0,0.0,-0.025,262147
4,46.0,95.0,65.0,20.0,5.0,93.0,124.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.609946,0.854549,0.534089,68.271576,82.0,0.0,75.1,0.0,0.0,-0.025,262147


In [ ]:
df[df.Vaso>0][['Vaso']].describe()

,Vaso
count,462884.000000
mean,0.278135
std,0.437714
min,0.000100
25%,0.051000
50%,0.160000
75%,0.324500
max,51.601000


In [ ]:
df[df.Fluids>0][['Fluids']].describe()

,Fluids
count,1.245956e+06
mean,1.208356e+02
std,9.918970e+02
min,6.000000e-03
25%,1.000000e+01
50%,2.000000e+01
75%,9.990000e+01
max,1.000150e+06


## Discretizing Actions

### Normalize some columns

In [ ]:
temp=df.describe()
means=temp.loc['mean'].values
stds=temp.loc['std'].values

cols=[0,1,2,3,4,5,6,7,11,37,38,40]

df.iloc[:,cols]=(df.iloc[:,cols]-means[cols])/stds[cols]
print('Normalized')

Normalized


In [ ]:
means

array([ 5.97680844e+01,  8.65268912e+01,  7.86168041e+01,  2.06791009e+01,
        6.44170886e+00,  9.71415643e+01,  1.21511315e+02,  3.69600955e+01,
        1.21276598e+00,  2.09433591e+00,  3.81532571e-01,  1.30679688e+00,
       -1.32587129e-01,  2.47033868e-02,  3.43234954e-03,  1.89323164e-01,
       -2.30669752e-02, -1.20075913e-01, -1.81488828e-01, -1.22392416e-02,
       -6.37759038e-02,  1.61877502e-01,  1.45423881e-01,  1.12890419e-01,
        9.76008341e-02,  1.35763436e-01, -4.07678096e-01,  1.81965244e-01,
        1.58229578e-01, -5.26373330e-01, -5.83405959e-02,  1.75772221e-01,
        8.53373985e-02, -4.23018796e-02,  8.08761724e-01,  1.23542296e+00,
        6.42965810e-01,  6.87600413e+01,  6.53440726e+01,  5.84643249e-01,
        8.37737428e+01,  4.95818074e-02,  5.78952733e+01,  6.22244670e-02,
        2.50184230e+05])

In [ ]:
stds

array([1.41127915e+01, 1.71324684e+01, 1.60435516e+01, 6.04456389e+00,
       3.85008103e+00, 3.27706005e+00, 2.24704618e+01, 7.89101766e-01,
       1.10273039e+00, 1.36031177e+00, 9.51707717e-01, 1.50444278e+00,
       9.50887586e-01, 1.00413460e+00, 9.20852861e-01, 9.64146936e-01,
       6.96247963e-01, 8.04110624e-01, 8.04816858e-01, 9.55594122e-01,
       8.33892990e-01, 9.31975058e-01, 1.05610087e+00, 7.95282991e-01,
       4.27973302e-01, 4.14742445e-01, 3.36382760e-01, 3.68723800e-01,
       3.75602683e-01, 3.37196671e-01, 3.53584285e-01, 3.66109995e-01,
       4.22208406e-01, 3.53814500e-01, 2.02588617e-01, 4.17686047e-01,
       1.34842540e-01, 1.65489857e+00, 1.58998495e+01, 4.92783537e-01,
       2.59243694e+01, 2.13275699e-01, 6.89739067e+02, 1.26622855e+00,
       2.85776138e+04, 1.06566984e+00, 1.27930427e+00, 5.70986634e+00])

In [ ]:
df.head()

,DBP,HR,MBP,RR,SOFA,SpO2,SBP,TempC,cardio,cns,liver,renal,AG,BICARB,CREAT,CL,GLUC,HCT,HB,PLAT,K,Na,BUN,WBC,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,R,C,T,SV,age,M/F,weight,Vaso,Fluids,Rewards,Pat
0,-0.975575,1.837045,-1.035731,0.880278,-1.673136,-0.958653,-0.556789,-0.723869,0.0,0.0,0.0,-0.868625,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.463221,1.030419,0.419510,-1.858045,1.047553,0.0,-0.334579,0.0,0.0,-0.250,262147
1,-1.329863,1.428464,-1.285052,0.383965,-1.153666,-0.653502,-0.556789,-0.723869,1.0,0.0,0.0,-0.203927,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.461611,0.946364,0.446352,-1.324097,1.047553,0.0,-0.334579,0.0,0.0,-0.375,262147
2,-1.117290,1.253358,-1.035731,-0.277787,-0.374462,0.261953,-0.156264,-0.723869,1.0,0.0,0.0,1.790167,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.509343,0.863491,0.458074,-1.098627,1.047553,0.0,-0.334579,0.0,0.0,-0.025,262147
3,-1.329863,0.552933,-1.160392,-0.112349,-0.374462,-0.348350,-0.156264,-0.723869,1.0,0.0,0.0,1.790167,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.554594,0.871751,0.518681,-0.403291,1.047553,0.0,-0.334579,0.0,0.0,-0.025,262147
4,-0.975575,0.494564,-0.848740,-0.112349,-0.374462,-1.263805,0.110754,-0.723869,1.0,0.0,0.0,1.790167,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.609946,0.854549,0.534089,-0.295163,1.047553,0.0,-0.334579,0.0,0.0,-0.025,262147


In [ ]:
def get_actions(df):
  df['Fluids']=df['Fluids'].apply(lambda x:max(x,0))
  Vaso_cuts,vaso_bins=pd.cut(df['Vaso'],bins=[-1e-6,1e-8/2,0.15,np.inf],labels=False,retbins=True)

  df['Vaso_cuts']=Vaso_cuts.values
  Fluids_cuts,fluid_bins=pd.cut(df['Fluids'],bins=[-1e-6,6.000000e-03/2,5.000000e+01,np.inf],labels=False,retbins=True)

  df['Fluid_cuts']=Fluids_cuts.values
  df['Fluid_cuts'].value_counts(normalize=True),df['Vaso_cuts'].value_counts(normalize=True)

  all_acts=np.arange(9).reshape(3,3)


  df['action']=all_acts[df.Vaso_cuts,df.Fluid_cuts]


get_actions(df)

### Data set class for Discrete RL

In [ ]:
class Discrete_RL_dataset(Dataset):
  """
   Need the df Ready and scaled
   Normalization is Done before

  """

  def __init__(self,scale_rewards=False,df=df):
    self.scale_rewards=scale_rewards
    self.df=df

  def __len__(self):
    return df.shape[0]

  def __getitem__(self,idx):
    
    temp=self.df.iloc[idx,:]
    # Rewards is always +- 15 at the terminal step and 
    done=int(np.abs(temp['Rewards'])==15)
    states=torch.FloatTensor(temp.iloc[:41].values).to(device)
    
    assert states.shape==(41,)
    if done:
      next_states=torch.zeros_like(states).to(device)
    
    else:
      next_states=torch.FloatTensor(self.df.iloc[idx+1,:].values[:41]).to(device)

    assert next_states.shape==(41,)

    reward=temp['Rewards']
    
    if self.scale_rewards:
      reward=(rewards-2.313230e-02)/9.304778e-01
    
    action=int(temp.action)

    # assert actions.shape==(2,)

    return states,next_states,action,reward,done

ds=Discrete_RL_dataset()
loader=DataLoader(ds,batch_size=64*2,shuffle=True)    

##### Weighted Loader

In [ ]:
def weight_func(x):
  if x==-15:
    return 80

  if x==15:
     return 5
  
  return 2



weights1=df['Rewards'].copy()
weights1=weights1.apply(weight_func)



weights2=df['Rewards'].iloc[1:].copy()
weights2=weights2.apply(lambda x: 5 if np.abs(x)==15 else 2)
weights2=np.hstack([np.abs(weights2).values ,np.array([10])])



weights3=df['Rewards'].iloc[5:].copy()
weights3=weights3.apply(lambda x: 5 if np.abs(x)==15 else 2)
weights3=np.hstack([np.abs(weights3).values ,np.array([2]*5)])

weights=weights1+weights2+weights3

sampler = torch.utils.data.sampler.WeightedRandomSampler(torch.DoubleTensor(weights),num_samples=len(weights))
new_loader =DataLoader(ds, batch_size=100,sampler=sampler)


## Distributional

In [ ]:

class DistributionalDQN(nn.Module):
    def __init__(self, state_dim, n_actions, N_ATOMS):
        super(DistributionalDQN, self).__init__()

        self.input_layer=nn.Linear(state_dim,512)
        self.n_atoms=N_ATOMS
        self.hiddens=nn.ModuleList([ nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU()) for _ in range(5) ])
        
       
        self.out=nn.Linear(512, n_actions * N_ATOMS)
       
  

    def forward(self, state):
        batch_size =state.size()[0]
        out=self.input_layer(state)

        for layer in self.hiddens:
          out=layer(out)
        
        out=self.out(out)
        
        return F.softmax(out.view(batch_size, -1, self.n_atoms),dim=2),F.log_softmax(out.view(batch_size, -1,self.n_atoms), dim = -1)


In [ ]:
class dist_DQN(object):
  def __init__(self,
               num_actions=9,
               state_dim=41,
               v_max=20,
               v_min=-20,
               device='cpu',
               gamma=0.999,
               tau=0.005,
               n_atoms=51
               ):
    self.device=device
    self.Q =DistributionalDQN(state_dim, num_actions,n_atoms).to(self.device)
    self.Q_target=copy.deepcopy(self.Q)
    self.optimizer=torch.optim.Adam(self.Q.parameters(),lr=1e-5)

    self.tau = tau
    self.gamma=gamma
    self.v_min=v_min
    self.v_max=v_max

    self.num_actions = num_actions
    self.atoms=n_atoms
    self.supports = torch.linspace(self.v_min, self.v_max, self.atoms).view(1, 1, self.atoms).to(self.device)
    self.delta = (self.v_max - self.v_min) / (self.atoms - 1)



  def train_epoch(self,data_loader,it=0):

    sum_q_loss=0
    for i,(state,next_state,action,reward,done) in enumerate(data_loader):

      batch_size=state.shape[0]
      action=torch.LongTensor(action.reshape(batch_size,1)).to(device)
      reward=torch.FloatTensor(reward.reshape(batch_size,1).cpu().numpy()).to(device)
      done=done.reshape(batch_size,1).to(device)

      batch=(state,next_state,action,reward,done)
      loss=self.compute_loss(batch)
      # print(loss.item())
      sum_q_loss+=loss.item()
       
	    
      if i%25==0:
        print('Epoch :',it,'Batch :', i,'Average Loss :',sum_q_loss/(i+1))


      self.optimizer.zero_grad()
      loss.backward()

      self.optimizer.step()
      self.polyak_target_update()


  def polyak_target_update(self):
     for param, target_param in zip(self.Q.parameters(), self.Q_target.parameters()):
        target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


  def compute_loss(self,batch):

     state,next_state,action,reward,done=batch  
     batch_size=state.shape[0]
     range_batch = torch.arange(batch_size).long().to(device)
     action = action.long()

     z_dist = torch.from_numpy(np.array([[self.v_min + i*self.delta for i in range(self.atoms)]]*batch_size)).to(device)
     z_dist = torch.unsqueeze(z_dist, 2).float() 

     _, log_Q_dist_prediction = self.Q(state)
     log_Q_dist_prediction = log_Q_dist_prediction[range_batch, action.squeeze(1), :]

     with torch.no_grad():
        Q_dist_target, _ = self.Q_target(next_state)
 
     Q_target = torch.matmul(Q_dist_target, z_dist).squeeze(1)
     a_star = torch.argmax(Q_target, dim=1)
     Q_dist_star = Q_dist_target[range_batch, a_star.squeeze(1),:]

     m = torch.zeros(batch_size,self.atoms).to(device)

     for j in range(self.atoms):
         T_zj = torch.clamp(reward + self.gamma * (1-done) * (self.v_min + j*self.delta), min = self.v_min, max = self.v_max)
            
         bj = (T_zj - self.v_min)/self.delta
         l = bj.floor().long()
         u = bj.ceil().long()

         mask_Q_l = torch.zeros(m.size()).to(device)
         mask_Q_l.scatter_(1, l, Q_dist_star[:,j].unsqueeze(1))
            
         mask_Q_u = torch.zeros(m.size()).to(device)
         mask_Q_u.scatter_(1, u, Q_dist_star[:,j].unsqueeze(1))
         m += mask_Q_l*(u.float() + (l == u).float()-bj.float())
         m += mask_Q_u*(-l.float()+bj.float())

     loss = - torch.sum(torch.sum(torch.mul(log_Q_dist_prediction, m),-1),-1) / batch_size
     return loss


  def get_action(self,state):
     with torch.no_grad():
      batch_size=state.shape[0]

      z_dist = torch.from_numpy(np.array([[self.v_min + i*self.delta for i in range(self.atoms)]]*batch_size)).to(device)
      z_dist = torch.unsqueeze(z_dist, 2).float()

      Q_dist, _ = self.Q(state)
 
      Q_exp = torch.matmul(Q_dist, z_dist).squeeze(1)
      a_star = torch.argmax(Q_exp, dim=1)

      return a_star



In [ ]:
model=dist_DQN(v_min=-18,v_max=18)
checkpoint=torch.load('dist_noW11.pt')
model.Q.load_state_dict(checkpoint['Q_state_dict'])
model.Q_target.load_state_dict(checkpoint['Q_target_state_dict'])



<All keys matched successfully>

In [ ]:
cd dBCQ/scratch_NOV/dist

/content/drive/.shortcut-targets-by-id/1Dn0AjHyN50Kv8tcF0f2bQsdPbRYa_49h/sepsis3-cohort/dBCQ/scratch_NOV/dist


In [ ]:
model.optimizer.lr=1e-7

In [ ]:
loader=DataLoader(ds,batch_size=100,shuffle=True)

for it in range(14,100):
  model.train_epoch(loader,it)
  torch.save({
            'Q_state_dict': model.Q.state_dict(),
             'Q_target_state_dict': model.Q_target.state_dict(),
            
                
            },'dist_noW{}.pt'.format(it))

In [ ]:
loader=DataLoader(ds,batch_size=256,shuffle=False)

In [ ]:
def get_RLactions(model,loader):
    actions=[]
    for i,batch in enumerate(loader):
      state=batch[0]
      a_star=model.get_action(state)
      actions.append(a_star)
      print((i/len(loader)), 'Done')

    acts=[x.squeeze(1) for x in actions]
    return np.hstack(acts)

In [ ]:
acts=[x.squeeze(1) for x in actions]

In [ ]:
dBCQ_actions=np.hstack(acts)
df['dist_acts']=dBCQ_actions
pd.Series(dBCQ_actions).value_counts(normalize=True)

In [ ]:
all_acts=np.arange(9).reshape(3,3)

In [ ]:
acts1=df['dist_acts'].apply(lambda x:np.argwhere(all_acts==x))
acts1=np.vstack([x for x in acts1.values])
# acts_df['dist_Vaso_2']=acts1[:,0]
# acts_df['dist_Fluids_2']=acts1[:,1]

In [ ]:
acts_df=df[['Fluid_cuts','Vaso_cuts','dist_Vaso','dist_Fluids']]

In [ ]:
act_df=pd.read_csv('actions.csv')

In [ ]:
# act_df.to_csv('actions.csv')

In [ ]:
df2=pd.read_csv('Bootstrap_actions.csv')

In [ ]:
df2.head()

,action,"('Act_boot_{}', '0')",Act_boot_1,Act_boot_2,Act_boot_3,noW_1,noW_2,noW_3,noW_4,Act_boot_23
0,0,2,1,2,2,2,2,2,1,2
1,0,2,1,1,2,2,2,1,0,2
2,0,0,1,1,0,2,2,2,0,2
3,0,0,1,1,2,2,2,2,0,2
4,0,0,1,1,2,2,2,2,0,2


### Checking Bootstrapped Actions

In [ ]:
def load_model(model,PATH):
  checkpoint=torch.load(PATH)
  model.Q.load_state_dict(checkpoint['Q_state_dict'])

In [ ]:
def get_RLactions(model,loader,k=0):
    actions=[]
    for i,batch in enumerate(loader):
      state=batch[0]
      a_star=model.get_action(state)
      
      actions.append(a_star)
      print(k,' ',(i/len(loader)), 'Done')

    acts=[x.squeeze(1) for x in actions]
    return np.hstack(acts)

In [ ]:
ls

In [ ]:
# df2=df[['action']]

In [ ]:
filenames=['dist_bootstrap_{}.pt'.format(i) for i in range(29)]
# ce_losses=[]
# kl_divs=[]
for k in range(5,29):
  
  loader=DataLoader(ds,batch_size=128,shuffle=False,num_workers=4)
  model_new=dist_DQN(v_min=-18,v_max=18)
  print('K =',k)
  load_model(model_new,filenames[k])
  acts=get_RLactions(model_new,loader,k)
  df2['Act_boot_{}'.format(k)]=acts
  df2.to_csv('Bootstrap_actions.csv',index=False)

# df2.to_csv('Bootstrap_actions.csv',index=False)

In [ ]:
ls dBCQ/scratch_NOV/dist

actions2.csv        dist_noW13.pt  dist_v2_0.pt   dist_v2_9.pt   dist_VAL1.pt
actions.csv         dist_noW1.pt   dist_v2_10.pt  dist_VAL10.pt  dist_VAL2.pt
dist_17_atoms_1.pt  dist_noW2.pt   dist_v2_11.pt  dist_VAL11.pt  dist_VAL3.pt
dist_17_atoms_2.pt  dist_noW3.pt   dist_v2_2.pt   dist_VAL12.pt  dist_VAL4.pt
dist_17_atoms_3.pt  dist_noW4.pt   dist_v2_3.pt   dist_VAL13.pt  dist_VAL5.pt
dist_17_atoms_4.pt  dist_noW5.pt   dist_v2_4.pt   dist_VAL14.pt  dist_VAL6.pt
dist_17_atoms_5.pt  dist_noW6.pt   dist_v2_5.pt   dist_VAL15.pt  dist_VAL7.pt
dist_noW10.pt       dist_noW7.pt   dist_v2_6.pt   dist_VAL16.pt  dist_VAL8.pt
dist_noW11.pt       dist_noW8.pt   dist_v2_7.pt   dist_VAL17.pt  dist_VAL9.pt
dist_noW12.pt       dist_noW9.pt   dist_v2_8.pt   dist_VAL18.pt  images/


In [ ]:
df2['noW_4'].value_counts(normalize=True)

0    0.590513
4    0.352971
1    0.034888
6    0.016615
5    0.002379
8    0.002224
7    0.000341
2    0.000058
3    0.000012
Name: noW_4, dtype: float64

In [ ]:
filenames=['dBCQ/scratch_NOV/dist/dist_noW{}.pt'.format(i) for i in range(12)]
# ce_losses=[]
# kl_divs=[]
for k in range(5,12):
  
  loader=DataLoader(ds,batch_size=256,shuffle=False,num_workers=6)
  model_new=dist_DQN(v_min=-18,v_max=18)
  print('K =',k)
  load_model(model_new,filenames[k])
  acts=get_RLactions(model_new,loader,k)
  df2['noW_{}'.format(k)]=acts
  df2.to_csv('Bootstrap_actions.csv',index=False)

df2.to_csv('Bootstrap_actions.csv',index=False)

### Fitting a Validation Value Distribution

In [ ]:
val_df=pd.read_csv('../../../RL_val.csv')
val_df=val_df.drop(['Unnamed: 0'],axis=1)

val_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,pat
0,57.0,90.0,69.0,14.0,5.0,100.0,98.0,37.799999,1.0,4.0,0.0,0.0,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.666731,1.667938,0.603311,67.941063,47.0,0.0,87.2,0.0,500.0,-0.025,200009
1,56.0,91.0,68.0,19.0,5.0,100.0,96.0,37.900002,1.0,4.0,0.0,0.0,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.664922,1.695960,0.606689,67.864685,47.0,0.0,87.2,0.0,375.0,-0.025,200009
2,55.0,93.0,67.0,17.0,5.0,100.0,86.0,38.000000,1.0,4.0,0.0,0.0,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.632300,2.194721,0.605790,67.283615,47.0,0.0,87.2,0.0,500.0,-0.025,200009
3,58.0,105.0,71.0,18.0,5.0,100.0,93.0,38.099998,1.0,4.0,0.0,0.0,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.611805,1.945178,0.534462,66.394684,47.0,0.0,87.2,0.0,500.0,-0.025,200009
4,57.0,96.0,69.0,16.0,5.0,100.0,99.0,37.900002,1.0,4.0,0.0,0.0,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.648374,1.579956,0.576718,67.517929,47.0,0.0,87.2,0.0,875.0,-0.025,200009


In [ ]:
val_df.iloc[:,cols]=(val_df.iloc[:,cols]-means[cols])/stds[cols]
print('Normalized')

Normalized


In [ ]:
val_df.columns='DBP	HR	MBP	RR	SOFA	SpO2	SBP	TempC	cardio	cns	liver	renal	AG	BICARB	CREAT	CL	GLUC	HCT	HB	PLAT	K	Na	BUN	WBC	l_1	l_2	l_3	l_4	l_5	l_6	l_7	l_8	l_9	l_10	R	C	T	SV	age	M/F	weight	Vaso	Fluids	Rewards	Pat'.split()
val_df.head()

,DBP,HR,MBP,RR,SOFA,SpO2,SBP,TempC,cardio,cns,liver,renal,AG,BICARB,CREAT,CL,GLUC,HCT,HB,PLAT,K,Na,BUN,WBC,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,R,C,T,SV,age,M/F,weight,Vaso,Fluids,Rewards,Pat
0,-0.196140,0.202721,-0.599419,-1.104976,-0.374462,0.872256,-1.046321,1.064380,1.0,4.0,0.0,-0.868625,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.666731,1.667938,0.603311,-0.494881,-1.153726,0.0,0.132164,0.0,500.0,-0.025,200009
1,-0.266998,0.261090,-0.661749,-0.277787,-0.374462,0.872256,-1.135327,1.191109,1.0,4.0,0.0,-0.868625,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.664922,1.695960,0.606689,-0.541034,-1.153726,0.0,0.132164,0.0,375.0,-0.025,200009
2,-0.337856,0.377827,-0.724079,-0.608663,-0.374462,0.872256,-1.580355,1.317833,1.0,4.0,0.0,-0.868625,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.632300,2.194721,0.605790,-0.892155,-1.153726,0.0,0.132164,0.0,500.0,-0.025,200009
3,-0.125282,1.078251,-0.474758,-0.443225,-0.374462,0.872256,-1.268835,1.444558,1.0,4.0,0.0,-0.868625,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.611805,1.945178,0.534462,-1.429307,-1.153726,0.0,0.132164,0.0,500.0,-0.025,200009
4,-0.196140,0.552933,-0.599419,-0.774101,-0.374462,0.872256,-1.001818,1.191109,1.0,4.0,0.0,-0.868625,-0.708817,-0.838437,-0.659904,1.777689,-0.618447,-0.213809,-0.107123,-0.724009,-0.344044,0.857453,-0.620443,0.128732,0.197923,0.105806,-0.255739,0.534732,0.483859,-0.758225,-0.380859,-0.082806,-0.122179,0.080003,0.648374,1.579956,0.576718,-0.750567,-1.153726,0.0,0.132164,0.0,875.0,-0.025,200009


In [ ]:
def get_actions(df):
  df['Fluids']=df['Fluids'].apply(lambda x:max(x,0))
  Vaso_cuts,vaso_bins=pd.cut(df['Vaso'],bins=[-1e-6,1e-8/2,0.15,np.inf],labels=False,retbins=True)

  df['Vaso_cuts']=Vaso_cuts.values
  Fluids_cuts,fluid_bins=pd.cut(df['Fluids'],bins=[-1e-6,6.000000e-03/2,5.000000e+01,np.inf],labels=False,retbins=True)

  df['Fluid_cuts']=Fluids_cuts.values
  df['Fluid_cuts'].value_counts(normalize=True),df['Vaso_cuts'].value_counts(normalize=True)

  all_acts=np.arange(9).reshape(3,3)


  df['action']=all_acts[df.Vaso_cuts,df.Fluid_cuts]

get_actions(val_df)

In [ ]:
def weight_func(x):
  if x==-15:
    return 80

  if x==15:
     return 5
  
  return 2



weights1=val_df['Rewards'].copy()
weights1=weights1.apply(weight_func)



weights2=val_df['Rewards'].iloc[1:].copy()
weights2=weights2.apply(lambda x: 5 if np.abs(x)==15 else 2)
weights2=np.hstack([np.abs(weights2).values ,np.array([10])])



weights3=val_df['Rewards'].iloc[5:].copy()
weights3=weights3.apply(lambda x: 5 if np.abs(x)==15 else 2)
weights3=np.hstack([np.abs(weights3).values ,np.array([2]*5)])

weights=weights1+weights2+weights3

sampler = torch.utils.data.sampler.WeightedRandomSampler(torch.DoubleTensor(weights),num_samples=len(weights))
ds1=Discrete_RL_dataset(df=val_df)
new_loader =DataLoader(ds1, batch_size=100,sampler=sampler)


In [ ]:
model=dist_DQN(v_min=-18,v_max=18)
for it in range(1,100):
  model.train_epoch(new_loader,it)
  torch.save({
            'Q_state_dict': model.Q.state_dict(),
             'Q_target_state_dict': model.Q_target.state_dict(),
            
                
            },'dist_VAL{}.pt'.format(it))

In [ ]:
len(new_loader)

1015

In [ ]:
val_df.shape

(101499, 48)